In [1]:
import numpy as np
import pandas as pd
import pyodbc
from sqlalchemy import create_engine
import datetime
import tkinter
from tkinter import ttk
from tkinter import Tk, Button
from tkcalendar import DateEntry
import subprocess


In [2]:
#log in to the server khu:khumbu
dbuser = 'dbuser'
dbpassword = 'mypassword'

#Define server conn parameters
server = 'servername\SQLEXPRESS'
database = 'database_name'
driver = 'SQL+Server+Native+Client+11.0'
username = str(dbuser)
password = str(dbpassword)

#Assign connection variable
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};\
                  SERVER='+server+';\
                  DATABASE='+database+';\
                  UID='+username+';\
                  PWD='+ password)


engine = create_engine('mssql+pyodbc://'
                      'username:password@servername\SQLEXPRESS/SB?'
                      'driver=ODBC Driver 17 for SQL Server'
                      )


In [3]:
#Queries
alltran = '''SELECT * from HistoryAll'''



def View_Transactions():
    df=pd.read_sql(alltran,engine)

    def showAlltrans():
        text.delete('1.0', 'end')  # Clear the text widget
        text.insert('end', df.to_string(index=False))  # Insert DataFrame content into the text widget



    root = tkinter.Tk()
    frame = tkinter.Frame(root)
    frame.pack()
    
    # Create a text widget to display the DataFrame
    text = tkinter.Text(root, wrap='none')
    text.pack(expand=True, fill='both')
    showAlltrans()
    
    frame.mainloop()


In [4]:
def update_rate():
    def rates_input():
        global ExchRate,CurrencyCode
        CurrencyCombo = RateCurrencyCombo.get()
        ExchRate=ExchRateEntry.get()
        CurrencyCode = "'"+CurrencyCombo+"'"
        ExchRate = "'"+ExchRate+"'"
        
        if RateCurrencyCombo.get() and ExchRateEntry.get() != "":
            cursor = cnxn.cursor()
            add_rate = '''INSERT INTO Rates (CurrencyCode,ExchRate) VALUES('''+CurrencyCode+''','''+ExchRate+''')'''
            update_rates = '''UPDATE Rates  SET ExchRate ='''+ExchRate+''' WHERE CurrencyCode = '''+CurrencyCode
            cursor.execute(update_rates)
            cursor.commit()
            cursor.close()
             # Display an saved message
            error_label.config(text="saved successifully",fg="green") 
        else:
             # Display an error message
            error_label.config(text="New rate was not saved",fg="red")
    
    RatesWindow=tkinter.Tk()

    #create Currency label
    Currencylabel = tkinter.Label(RatesWindow,text='Currency')
    Currencylabel.grid(row=0,column=0,padx=5,pady=10,sticky="news")

    #create Currency Combo
    RateCurrencyCombo = ttk.Combobox(RatesWindow,values=['USD','ZIG','ZAR'],state='readonly')
    RateCurrencyCombo.grid(row=0,column=1,padx=5,pady=10,sticky="news")
    
    #create Account Name label
    ExchRateLabel = tkinter.Label(RatesWindow,text='Exch Rate')
    ExchRateLabel.grid(row=1,column=0,padx=5,pady=10,sticky="news")
    
    #create create Account Name Entry
    ExchRateEntry = tkinter.Entry(RatesWindow)
    ExchRateEntry.grid(row=1,column=1,padx=5,pady=10,sticky="news")

     # Create a label to display error message
    error_label = tkinter.Label(RatesWindow, fg="red")
    error_label.grid(row=3,column=0,pady=10,sticky="news")
    
    #create save button
    savebutton = tkinter.Button(RatesWindow,text='Save and Next', command=rates_input,bg="orange")
    savebutton.grid(row=2,column=0,padx=5,pady=10,sticky="news",columnspan=10)

    RatesWindow.mainloop()




In [5]:


# In[11]:
#open new account

 #New Account Input
def open_acc():
    def new_account():
        #Define Entry variables
        AccName = AccNameEntry.get()
        AccCurrency = CurrencyCombo.get()
        AccID = AccIDEntry.get()
        AccClass = AccClassCombo.get()
        AccGroup = AccGroupCombo.get()
        
        
        
        AccName = "'"+AccName+"'"
        AccCurrency = "'"+AccCurrency+"'"
        AccID = "'"+AccID+"'"
        AccClass = "'"+AccClass+"'"
        AccGroup = "'"+AccGroup+"'"
        
        if AccNameEntry.get() and AccIDEntry.get() and CurrencyCombo.get() and AccClassCombo.get() and AccGroupCombo.get() != "":
            cursor = cnxn.cursor()
        
        
            add_acc = '''INSERT INTO Accounts (AccID,AccName,AccClass,AccGroup,AccCurrency,CreatedOn) VALUES ('''+AccID+''','''+AccName+''','''+AccClass+''','''+AccGroup+''','''+AccCurrency+''',GETDATE());'''
        
            cursor.execute(add_acc)
            cursor.commit()
            cursor.close()
        
            AccNameEntry.delete(0, tkinter.END)
            AccIDEntry.delete(0, tkinter.END)
            
             # Display an saved message
            error_label.config(text="saved successifully",fg="green") 
            
        else:
             # Display an error message
            error_label.config(text="New Account was not saved",fg="red") 
    
    
    
    #create Account creation form
    AccWindow = tkinter.Tk()
    AccWindow.title('New Account Details')
    AccWindow.geometry('800x350')
    
    #create Currency label
    Currencylabel = tkinter.Label(AccWindow,text='Currency')
    Currencylabel.grid(row=4,column=0,padx=5,pady=10,sticky="news")

    #create Currency Combo
    CurrencyCombo = ttk.Combobox(AccWindow,values=['USD','ZIG','ZAR'],state='readonly')
    CurrencyCombo.grid(row=4,column=1,padx=5,pady=10,sticky="news")
    
    #create Account Name label
    AccNameLabel = tkinter.Label(AccWindow,text='Account Name')
    AccNameLabel.grid(row=0,column=0,padx=5,pady=10,sticky="news")
    
    #create create Account Name Entry
    AccNameEntry = tkinter.Entry(AccWindow)
    AccNameEntry.grid(row=0,column=1,padx=5,pady=10,sticky="news")
    
    #create Account ID label
    AccIDLabel = tkinter.Label(AccWindow,text='Account ID')
    AccIDLabel.grid(row=1,column=0,padx=5,pady=10,sticky="news")
    
    #create create Account Name Entry
    AccIDEntry = tkinter.Entry(AccWindow)
    AccIDEntry.grid(row=1,column=1,padx=5,pady=10,sticky="news")
    
    
        #create Account Class label
    AccClass = tkinter.Label(AccWindow,text='Account Class')
    AccClass.grid(row=2,column=0,padx=5,pady=10,sticky="news")
    
    #create Account Class Combo
    AccClassCombo = ttk.Combobox(AccWindow,values=['Cash','Banks','Inventory','Receivables','Payable','Property and Equip','Profit and loss'],state='readonly')
    AccClassCombo.grid(row=2,column=1,padx=5,pady=10,sticky="news")
    
    #create Account Group label
    AccGroup = tkinter.Label(AccWindow,text='Account Group')
    AccGroup.grid(row=3,column=0,padx=5,pady=10,sticky="news")
    
    #create Account Group Combo
    AccGroupCombo = ttk.Combobox(AccWindow,values=['Assets','Liabilities','Equity','Operating Expenditure','Income','Cost of Sales'],state='readonly')
    AccGroupCombo.grid(row=3,column=1,padx=5,pady=10,sticky="news",columnspan=200)
    
    #create save button
    savebutton = tkinter.Button(AccWindow,text='Save and Next', command=new_account,bg="orange")
    savebutton.grid(row=5,column=0,padx=5,pady=10,sticky="news",columnspan=10)
    
    # Create a label to display error message
    error_label = tkinter.Label(AccWindow, fg="red")
    error_label.grid(row=6,column=0,pady=10,sticky="news")


    AccWindow.mainloop()


##
def tran_reversal():
    #Define Entry variables
    TranID = TranIDEntry.get()
    RevDetails=RevDetailsEntry.get()

    TranID = "'"+TranID+"'"
    RevDetails = "'"+RevDetails+"'"

    
    #establish conn to the SQLEXP Server
    
    cursor = cnxn.cursor()

    #insert reversal entry
    revquery='''INSERT INTO HistoryAll (Date,Invoice,DebitAccNumber,CreditAccNumber,Details,CurrencyCode,CurrencyAmount,Rate,Amount,CreationDate) SELECT Date,Invoice,CreditAccNumber,DebitAccNumber,CONCAT('Reversal of',TranID),CurrencyCode,CurrencyAmount,Rate,Amount,GETDATE() FROM HistoryAll WHERE Details NOT LIKE '%Reversal%' AND TranID = '''+TranID

    cursor.execute(revquery)
    cursor.commit()
    cursor.close()
    TranIDEntry.delete(0, tkinter.END)
    RevDetailsEntry.delete(0, tkinter.END)
    
    
def rev_input():
    global TranIDEntry,RevDetailsEntry
    #initialise window
    reversalWindow = tkinter.Tk() 
    reversalWindow.title('Enter Transaction details')
    
    #create Reference label
    TranIDlabel = tkinter.Label(reversalWindow,text='Internal transaction ID')
    TranIDlabel.grid(row=0,column=0,padx=20,pady=5,sticky="news")
    
    #create date Entry
    TranIDEntry = tkinter.Entry(reversalWindow)
    TranIDEntry.grid(row=1,column=0,padx=20,pady=5,sticky="news")
    
    #create TranType label
    RevDetailslabel = tkinter.Label(reversalWindow,text='Enter reason for reversal')
    RevDetailslabel.grid(row=0,column=1,padx=20,pady=5,sticky="news")
    
    #create TranType Combo
    RevDetailsEntry = tkinter.Entry(reversalWindow)
    RevDetailsEntry.grid(row=1,column=1,padx=20,pady=5,sticky="news")

     #create save button
    Confirmbutton = tkinter.Button(reversalWindow,text='Confirm', command=tran_reversal,bg="orange")
    Confirmbutton.grid(row=2,column=0,padx=20,pady=5,sticky="news")
  
    reversalWindow.mainloop()
    


# In[13]:





# In[14]:


def AddTransaction():
    global Currency
    #Define Entry variables
    Date = DateEntrybox.get()
    Invoice = InvoiceEntry.get()
    DebitAccNumber = DebitAccEntry.get()
    CreditAccNumber = CreditAccEntry.get()
    Rate = RateEntry.get()
    Phone_number = CustomerPhoneEntry.get()
    Transaction_type = TranTypeCombo.get()
    Currency = CurrencyCombo.get() 
    CurrencyAmount = AmountEntry.get()
    Details = DetailsEntry.get()
    


    
    Date = "'"+Date+"'"
    Invoice = "'"+Invoice+"'"
    Rate = "'"+Rate+"'"
    DebitAccNumber = "'"+DebitAccNumber+"'"
    CreditAccNumber = "'"+CreditAccNumber+"'"
    Phone_number = "'"+Phone_number+"'"
    Transaction_type ="'"+Transaction_type+"'"
    Currency = "'"+Currency+"'"
    CurrencyAmount = "'"+CurrencyAmount+"'"
    Details = "'"+Details+"'"

    
    if DateEntrybox.get() and DebitAccEntry.get() and CreditAccEntry.get() and RateEntry.get() and CurrencyCombo.get()and AmountEntry.get() != "":
        #establish conn to the SQLEXP Server
        cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};\
                      SERVER='+server+';\
                      DATABASE='+database+';\
                      UID='+username+';\
                      PWD='+ password)
        cursor = cnxn.cursor()
        Add_tran = '''INSERT INTO HistoryAll (Date,Invoice,DebitAccNumber,CreditAccNumber,Details,CurrencyCode,Currencyamount,Rate,CreationDate)
        VALUES ('''+Date+''','''+Invoice+''','''+DebitAccNumber+''','''+CreditAccNumber+''','''+Details+''','''+Currency+''','''+CurrencyAmount+''','''+Rate+''','''+'''GETDATE());
        '''
        # Update converted amount
        updateAmount = '''UPDATE HistoryAll SET Amount=CurrencyAmount*Rate'''

        cursor.execute(Add_tran)
        cursor.commit()
        cursor.execute(updateAmount)
        cursor.commit()
        cursor.close()
        DebitAccEntry.delete(0, tkinter.END)
        CreditAccEntry.delete(0, tkinter.END)
        AmountEntry.delete(0, tkinter.END)
        RateEntry.delete(0, tkinter.END)
        
         #start progress bar
        progressbar.stop()
        # Display an error message
        error_label.config(text="Transaction with REF "+Invoice+" was succesifully captured",fg="green") 

    else:
        # Display an error message
        error_label.config(text="Please enter all details",fg="red") 

# query accounts table to get values for combo

Acclistdf1 = ['Cash_Main','BancABC ZIG']

#Add search fn to Debit combo
def acc_search(event):
    
    searchterm=DebitAccEntry.get().lower()
    filteredItems=[item for item in accliststr if searchterm in item.lower()]
    DebitAccEntry['values']=filteredItems
    # credit acc entry
def acc_search_cr(event):

    searchterm=CreditAccEntry.get().lower()
    filteredItems=[item for item in accliststr if searchterm in item.lower()]
    DebitAccEntry['values']=filteredItems





    


cursor = cnxn.cursor()
accquery = '''SELECT AccName FROM Accounts'''
cursor.execute(accquery)
acclist1 = cursor.fetchall()
cursor.close()

#create combo list
accliststr=str([acclist1]).strip().replace("[","").replace("(","").replace("),","").replace(",)]]","").replace("'","").split(", ")

#create windows form
window = tkinter.Tk()
window.title('Transaction Details')
window.geometry('800x350')


# create a toplevel menu
menubar = tkinter.Menu(window)

# create a pulldown menu, and add it to the menu bar
filemenu = tkinter.Menu(menubar)
filemenu.add_command(label="New Transaction", command=None)
filemenu.add_command(label="Transaction reversal", command=rev_input)
filemenu.add_command(label="Update Rate", command=update_rate)


Reportsmenu = tkinter.Menu(menubar)
Reportsmenu.add_command(label="Cash book summary", command=None)
Reportsmenu.add_command(label="Reversal", command=None)
Reportsmenu.add_command(label="View Transactions", command=View_Transactions)

AccountsMenu = tkinter.Menu(menubar)
AccountsMenu.add_command(label="Open Account", command=open_acc)
AccountsMenu.add_command(label="Accounts list", command=None)

menubar.add_cascade(label="Options", menu=filemenu)
menubar.add_cascade(label="Reports", menu=Reportsmenu)
menubar.add_cascade(label="Accounts", menu=AccountsMenu)
window.config(menu=menubar)

                     
#create Accs details label frame
AccountDetailsFrame = tkinter.LabelFrame(window,text="Enter Details",fg="blue")
AccountDetailsFrame.grid(row=0,column=0,padx=80,sticky="news")

#create date label
Datelabel = tkinter.Label(AccountDetailsFrame,text='Date')
Datelabel.grid(row=0,column=0,padx=20,pady=5,sticky="news")

# Create a DateEntry widget
DateEntrybox = DateEntry(AccountDetailsFrame, date_pattern="yyyy-mm-dd")
DateEntrybox.grid(row=1,column=0,padx=10, pady=10)

#create TranType label
TranTypelabel = tkinter.Label(AccountDetailsFrame,text='Transaction Type')
TranTypelabel.grid(row=0,column=1,padx=20,pady=5,sticky="news")


#create TranType Combo
TranTypeCombo = ttk.Combobox(AccountDetailsFrame,values=['Pay_Out','Receive_money'])
TranTypeCombo.grid(row=1,column=1,padx=20,pady=5,sticky="news")

#create Currency label
Currencylabel = tkinter.Label(AccountDetailsFrame,text='Currency')
Currencylabel.grid(row=0,column=2,padx=20,pady=5,sticky="news")

#create Currency Combo
CurrencyCombo = ttk.Combobox(AccountDetailsFrame,values=['USD','ZIG','ZAR'])
CurrencyCombo.grid(row=1,column=2,padx=20,pady=5,sticky="news")


#create Reference label
Invoicelabel = tkinter.Label(AccountDetailsFrame,text='Invoice Number')
Invoicelabel.grid(row=2,column=0,padx=20,pady=5,sticky="news")


#create Reference Entry box
InvoiceEntry = tkinter.Entry(AccountDetailsFrame)
InvoiceEntry.grid(row=3,column=0,padx=20,pady=5,sticky="news")


#create Amount label
Amountlabel = tkinter.Label(AccountDetailsFrame,text='Amount')
Amountlabel.grid(row=2,column=1,padx=20,pady=5,sticky="news")


#create Amount Entrybox
AmountEntry = tkinter.Entry(AccountDetailsFrame)
AmountEntry.grid(row=3,column=1,padx=20,pady=5,sticky="news")


#create LabelFrame for Customer Details

CustomerDetailsFrame = tkinter.LabelFrame(window,text="Accounts Details",fg="blue")
CustomerDetailsFrame.grid(row=1,column=0,padx=80,sticky="news")


#create Debit Acc label
DebitAcclabel = tkinter.Label(CustomerDetailsFrame,text='Debit Account')
DebitAcclabel.grid(row=0,column=0,padx=20,pady=5,sticky="news")

#create Debit Acc Entrybox
DebitAccEntry = ttk.Combobox(CustomerDetailsFrame,values = accliststr)
DebitAccEntry.grid(row=1,column=0,padx=20,pady=5,sticky="news")
DebitAccEntry.bind("<KeyRelease>",acc_search)

#create Credit Acc label
CreditAcclabel = tkinter.Label(CustomerDetailsFrame,text='Credit Account')
CreditAcclabel.grid(row=0,column=1,padx=20,pady=5,sticky="news")

#create Credit Acc Entry
CreditAccEntry = ttk.Combobox(CustomerDetailsFrame,values = accliststr)
CreditAccEntry.grid(row=1,column=1,padx=20,pady=5,sticky="news")
CreditAccEntry.bind("<KeyRelease>",acc_search_cr)





#create Rate Entry
def currency_rate_insert():
    currCode = CurrencyCombo.get()
    currCode = "'"+currCode+"'"
    cursor=cnxn.cursor()
    rateQuery='''SELECT ExchRate from Rates WHERE CurrencyCode ='''+currCode
    cursor.execute(rateQuery)
    currRate = cursor.fetchall()
    cursor.close()
    RateEntry.insert(0,str(currRate).replace("[(","").replace(",)]",""))

#create Rate label
RateUpdateButton = tkinter.Button(CustomerDetailsFrame,text='Update Exch rate',command=currency_rate_insert)
RateUpdateButton.grid(row=0,column=2,padx=20,pady=5,sticky="news")

   
RateEntry = tkinter.Entry(CustomerDetailsFrame)
RateEntry.grid(row=1,column=2,padx=20,pady=5,sticky="news")



#create Phone label
CustomerPhonelabel = tkinter.Label(CustomerDetailsFrame,text='Phone Number')
CustomerPhonelabel.grid(row=2,column=0,padx=20,pady=5,sticky="news")

#create Phone Entry
CustomerPhoneEntry = tkinter.Entry(CustomerDetailsFrame)
CustomerPhoneEntry.grid(row=3,column=0,padx=20,pady=5,sticky="news")

#create details label
Detailslabel = tkinter.Label(CustomerDetailsFrame,text='Other Details')
Detailslabel.grid(row=2,column=1,padx=20,pady=5,sticky="news")

#create Phone Entry
DetailsEntry = tkinter.Entry(CustomerDetailsFrame)
DetailsEntry.grid(row=3,column=1,padx=20,pady=5,sticky="news")

# Create a label to display error message
error_label = tkinter.Label(CustomerDetailsFrame, fg="red")
error_label.grid(row=5,column=0,pady=5,sticky="news")

#progress bar
progressbar=ttk.Progressbar(CustomerDetailsFrame,mode="indeterminate")
progressbar.grid(row=4,column=2,padx=20,pady=5,sticky="news")



#create save button
savebutton = tkinter.Button(CustomerDetailsFrame,text='Save and Next', command=AddTransaction,bg="orange")
savebutton.grid(row=4,column=0,padx=20,pady=5,sticky="news")

window.mainloop()

In [6]:
str(currRate).replace("[(","").replace(",)]","")

NameError: name 'currRate' is not defined